# Shapash: Python machine learning interpretable

- Criar um ambiente com python 3.10 <br>
conda create -n py310 python==3.10 <br>

- Ativar o ambiente com python 3.10 <br>
conda activate py310 <br>


- No Terminal de Comando: <br>
conda activate py310 <br>
pip install -q shapash        <br>
pip install -q shapash[report]  <br>

- instalar a IDE Jupyter notebook <br>
pip install -q jupyter <br>
python --version <br>
jupyter notebook

In [1]:
!python --version

Python 3.10.0


In [2]:
%%time

!pip install -q shapash

CPU times: user 26.9 ms, sys: 14.7 ms, total: 41.6 ms
Wall time: 2.06 s


# Carregamento dos dados

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
%%time
!pip install -q XGBoost

CPU times: user 28.3 ms, sys: 13.6 ms, total: 41.9 ms
Wall time: 1.94 s


In [5]:
!pip install -q lightgbm

In [6]:
# brew install libomp

In [7]:
from lightgbm import LGBMRegressor

In [8]:
from category_encoders import OrdinalEncoder
import numpy as np 
import pandas as pd 
import xgboost as xg 
from category_encoders import OrdinalEncoder
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.ensemble import ExtraTreesRegressor

# Building Supervized Model

In [9]:
from shapash.data.data_loader import data_loading
house_df, house_dict = data_loading('house_prices')

In [10]:
y_df=house_df['SalePrice'].to_frame()
X_df=house_df[house_df.columns.difference(['SalePrice'])]

In [11]:
house_df.shape

(1460, 73)

In [12]:
house_df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,2-Story 1946 & Newer,Residential Low Density,8450,Paved,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Inside lot,Gentle slope,College Creek,...,0,0,0,0,0,2,2008,Warranty Deed - Conventional,Normal Sale,208500
2,1-Story 1946 & Newer All Styles,Residential Low Density,9600,Paved,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Frontage on 2 sides of property,Gentle slope,Veenker,...,0,0,0,0,0,5,2007,Warranty Deed - Conventional,Normal Sale,181500
3,2-Story 1946 & Newer,Residential Low Density,11250,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Inside lot,Gentle slope,College Creek,...,0,0,0,0,0,9,2008,Warranty Deed - Conventional,Normal Sale,223500
4,2-Story 1945 & Older,Residential Low Density,9550,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Corner lot,Gentle slope,Crawford,...,272,0,0,0,0,2,2006,Warranty Deed - Conventional,Abnormal Sale,140000
5,2-Story 1946 & Newer,Residential Low Density,14260,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Frontage on 2 sides of property,Gentle slope,Northridge,...,0,0,0,0,0,12,2008,Warranty Deed - Conventional,Normal Sale,250000


# Encoding Categorical Features

In [13]:
from category_encoders import OrdinalEncoder

categorical_features = [col for col in X_df.columns if X_df[col].dtype == 'object']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(X_df)

X_df=encoder.transform(X_df)

# Train / Test Split

In [14]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, y_df, train_size=0.75, random_state=1)

# Model Fitting

In [15]:
regressor = xg.XGBRegressor(objective ='reg:linear', 
                  n_estimators = 10, seed = 123) 

# Understanding my model with shapash

## Declare and Compile SmartExplainer

In [16]:
from shapash import SmartExplainer

In [17]:
%%time
xpl = SmartExplainer(
    model=regressor,
    preprocessing=encoder,   # Optional: compile step can use inverse_transform method
    features_dict=house_dict # optional parameter, specifies label for features name 
)

TypeError: The passed model is not callable and cannot be analyzed directly with the given masker! Model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, objective='reg:linear', ...)

In [18]:
xpl.compile(x=Xtest,
            y_target=ytest # Optional: allows to display True Values vs Predicted Values
           )

NameError: name 'xpl' is not defined

In [ ]:
app = xpl.run_app(title_story='House Prices', port=8020)

In [ ]:
app.kill()

In [ ]:
import sklearn
from sklearn import linear_model

In [ ]:
%%time

regressor = linear_model
from shapash import SmartExplainer
xpl = SmartExplainer(
  model=regressor,
  features_dict=house_dict,  # Optional parameter
  preprocessing=encoder, # Optional: compile step can use inverse_transform method
  postprocessing=postprocess, # Optional: see tutorial postprocessing  
)

In [ ]:
%%time

xpl.compile(
    x=Xtest,    
    y_pred=y_pred, # Optional: for your own prediction (by default: model.predict)
    y_target=yTest, # Optional: allows to display True Values vs Predicted Values
    additional_data=X_additional, # Optional: additional dataset of features for Webapp
    additional_features_dict=features_dict_additional, # Optional: dict additional data    
)

In [ ]:
%%time

app = xpl.run_app()

In [ ]:
%%time

xpl.generate_report(
    output_file='path/to/output/report.html',
    project_info_file='path/to/project_info.yml',
    x_train=Xtrain,
    y_train=ytrain,
    y_test=ytest,
    title_story="House prices report",
    title_description="""This document is a data science report of the kaggle house prices tutorial project.
        It was generated using the Shapash library.""",
    metrics=[{'name': 'MSE', 'path': 'sklearn.metrics.mean_squared_error'}]
)